# Notebook to analyze and display scRNAseq data


### Load important modules

In [1]:
# Standard modules
import numpy as np
import os
import pandas as pd 
import plotly.graph_objects as go
from scipy import linalg

# Move to root directory for easier module handling
os.chdir("../..")
print(os.listdir("."))

#LBAE imports
from modules.maldi_data import MaldiData
from modules.figures import Figures
from modules.atlas import Atlas
from modules.launch import Launch
from modules.storage import Storage


# multithreading/multiprocessing
from multiprocessing import Pool
from threadpoolctl import threadpool_limits

# set thread limit
threadpool_limits(16)


['.git', '.gitattributes', '.vscode', 'LICENSE.md', 'README.md', 'TODO.py', 'app.py', 'assets', 'config.py', 'data_sample', 'documentation', 'index.py', 'js', 'main.py', 'modules', 'notebooks', 'pages', 'readme', 'requirements.txt', 'style', 'data', '__pycache__', '.gitignore']


#### Load LBAE objects

In [2]:
path_data = "data/whole_dataset/"
path_annotations = "data/annotations/"
path_db = "data/app_data/data.db"

# Load shelve database
storage = Storage(path_db)

# Load data
data = MaldiData(path_data, path_annotations)

# If True, only a small portions of the figures are precomputed (if precomputation has not already
# been done). Used for debugging purposes.
sample = False

# Load Atlas and Figures objects. At first launch, many objects will be precomputed and shelved in
# the classes Atlas and Figures.
atlas = Atlas(data, storage, resolution=25)
figures = Figures(data, storage, atlas)

allen mouse atlas (res. 25um)
From: http://www.brain-map.org (Wang et al 2020, https://doi.org/10.1016/j.cell.2020.04.007 )


### Load data

In [3]:
expr_table = pd.read_csv('notebooks/scRNAseq/data/expr_normalized_table.tsv', sep='\t')
meta_table = pd.read_csv('notebooks/scRNAseq/data/meta_table.tsv', sep='\t', usecols=[0, 2,3,4, 7] )

In [ ]:
# Reorganize columns
meta_table = meta_table[['Unnamed: 0', 'stereo_AP', 'stereo_DV', 'stereo_ML', 'ABA_acronym']]
meta_table

In [ ]:
counts = meta_table['ABA_acronym'].value_counts()

In [ ]:
# Get coordinates of all the uniques structures in the molecular atlas
l_x = []
l_y = []
l_z = []
l_xs = []
l_ys = []
l_zs = []
for acronym in counts[counts == 1].index.to_list():
    try:
        id_structure = atlas.bg_atlas.structures[acronym]['id']
        array_coor = np.where(atlas.bg_atlas.annotation == id_structure)
        xs, ys, zs = np.mean(array_coor, axis=1)*25/1000
        std_xs, std_ys, std_zs = np.std(array_coor, axis=1)*25/1000
        print(std_xs, std_ys, std_zs)
        x, y, z = meta_table[meta_table['ABA_acronym'] == acronym].iloc[0,1:4].to_numpy()
        if std_xs<0.1:
            l_x.append(x)
            l_xs.append(xs)
        if std_ys<0.1:
            l_y.append(y)
            l_ys.append(ys)
        if std_zs<0.1:
            l_z.append(z)
            l_zs.append(zs)
    except:
        pass
  
X = np.vstack([np.array(l_x), np.ones(len(l_x)), np.zeros(len(l_y)), np.zeros(len(l_z))   ]).T
Y = np.vstack([np.array(l_y), np.zeros(len(l_x)), np.ones(len(l_y)), np.zeros(len(l_z))    ]).T
Z = np.vstack([np.array(l_z), np.zeros(len(l_x)), np.zeros(len(l_y)) , np.ones(len(l_z))      ]).T
M = np.hstack((X,Y,Z))
y = np.array(l_xs + l_ys + l_zs)

a, b, c, d = np.linalg.lstsq(M, y, rcond=None)[0]

print(a, b, c, d)



In [ ]:
import matplotlib.pyplot as plt
_ = plt.plot(np.array(l_x), np.array(l_xs), 'o', label='Original data', markersize=10)
_ = plt.plot(np.array(l_x), a*np.array(l_x) + b, 'r', label='Fitted line')
_ = plt.plot(np.array(l_y), np.array(l_ys), 'o', label='Original data', markersize=10)
_ = plt.plot(np.array(l_y), a*np.array(l_y) + c, 'r', label='Fitted line')
_ = plt.plot(np.array(l_z), np.array(l_zs), 'o', label='Original data', markersize=10)
_ = plt.plot(np.array(l_z), a*np.array(l_z) + d, 'r', label='Fitted line')
_ = plt.legend()
plt.show()

In [ ]:
# Convert molecular atlas coordinates to our system of coordinates
meta_table["stereo_AP"] = a * meta_table["stereo_AP"] + b
meta_table["stereo_DV"] = a * meta_table["stereo_DV"] + c
meta_table["stereo_ML"] = a * meta_table["stereo_ML"] + d

In [ ]:
meta_table

In [ ]:
scatter = go.Scatter3d(
    x=meta_table['stereo_AP'].to_numpy(),
    y=meta_table['stereo_DV'].to_numpy(),
    z=meta_table['stereo_ML'].to_numpy(),
    mode='markers',
    marker=dict(
        size=1,
        opacity=0.8
    )
)

fig = go.Figure(data=scatter)

fig.show()

In [ ]:
# Get root figure
root_data = figures._storage.return_shelved_object(
    "figures/3D_page",
    "volume_root",
    force_update=False,
    compute_function=figures.compute_3D_root_volume,
)

In [ ]:
fig = go.Figure(data=[root_data, scatter])


# Hide grey background
fig.update_layout(
    margin=dict(t=0, r=0, b=0, l=0),
    scene=dict(
        xaxis=dict(backgroundcolor="rgba(0,0,0,0)"),
        yaxis=dict(backgroundcolor="rgba(0,0,0,0)"),
        zaxis=dict(backgroundcolor="rgba(0,0,0,0)"),
    ),
)

# Set background color to zero
fig.layout.template = "plotly_dark"
fig.layout.plot_bgcolor = "rgba(0,0,0,0)"
fig.layout.paper_bgcolor = "rgba(0,0,0,0)"


fig.show()